In [1]:
import pandas as pd

In [2]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
productos = pd.read_csv("../Productos2.csv", names=("product_id", "product_name", "price", "uses_credit"), encoding='latin1')
clientes = pd.read_csv("../Clientes2.csv", names=("client_id", "client_name", "phonenumber", "credit"))
pedidos = pd.read_csv("../Pedidos2.csv", names=("product_id", "client_id", "qty"))

clientes = clientes.set_index('client_id')
productos = productos.set_index('product_id')
# pedidos['date'] = pd.to_datetime(pedidos['date'], format="%d/%m/%Y")

In [4]:
productos.head()

,product_name,price,uses_credit
product_id,,,
BIT-434,Campana Extractora modelo Glass,375.09,S
SSE-115,Refrigeradora CoolStyle 311N Steel,"3,243.58",S
NMV-644,Lavadora Automatica,"3,272.48",S
MLE-193,Cocina a gas Ingenious,"2,779.41",S
YYK-309,Refrigeradora Door in Door,"4,079.44",S


In [5]:
clientes.head()

,client_name,phonenumber,credit
client_id,,,
79464412,PORTUGAL RAFFO ALEXANDER,3902394,10000
16552775,YALLICO PAREDES LOURDES CARMELA,960176666,20000
20864087,ZEGARRA CASANOVA ESTEFANIA,2639311,10000
94326707,CHANG SEGURA ANGEL,984185862,10000
73786035,ORTIZ EPIQUEN HENRY ISRAEL,6873908,20000


In [6]:
pedidos.head()

,product_id,client_id,qty
0,JXD-139,50375303,6
1,CRU-009,50375303,5
2,YYK-309,22777006,3
3,OTS-581,42157219,5
4,AWB-345,13245501,1


In [7]:
# reduce stock from products as orders come

pedidos['served'] = False
for i, pedido in pedidos.iterrows():
    cid = pedido['client_id']
    pid = pedido['product_id']
    qty = pedido['qty']
    credit = clientes.loc[cid, 'credit']
    price = productos.loc[pid, 'price']
    uses_credit = productos.loc[pid, 'uses_credit']
    total_cost = qty * price
    if uses_credit == 'S' and credit >= total_cost:
        clientes.loc[cid, 'credit'] -= total_cost
        pedidos.loc[i, 'served'] = True
        

In [32]:
# reporte

# # This creates a report with different order for clients (as they appear in pedidos) 
# for cid, df in pedidos.groupby('client_id', sort=False):
#     print('========================================================================================================================')
#     print(clientes.loc[[cid]])
#     print('------------------------------------------------------------------------------------------------------------------------')
#     print(df.loc[df['served'], ['product_id', 'client_id', 'qty']])

for i, client in clientes.reset_index().iterrows():
    print('========================================================================================================================')
    print(client)
    print('------------------------------------------------------------------------------------------------------------------------')
    mask = (pedidos['client_id'] == client['client_id']) & (pedidos['served'])
    t = pedidos.loc[mask, ['product_id', 'qty']]
    t = t.merge(productos, how='left', on='product_id')
    t['total'] = t['qty'] * t['price']
    print(t[['product_id', 'qty', 'total']])

client_id                      79464412
client_name    PORTUGAL RAFFO ALEXANDER
phonenumber                     3902394
credit                         5,782.30
Name: 0, dtype: object
------------------------------------------------------------------------------------------------------------------------
  product_id  qty    total
0    NSM-283    2 1,873.62
1    JXD-139    4 2,344.08
client_id                             16552775
client_name    YALLICO PAREDES LOURDES CARMELA
phonenumber                          960176666
credit                                3,716.33
Name: 1, dtype: object
------------------------------------------------------------------------------------------------------------------------
  product_id  qty     total
0    AWB-345    6  4,802.22
1    CNN-411    5 11,481.45
client_id                        20864087
client_name    ZEGARRA CASANOVA ESTEFANIA
phonenumber                       2639311
credit                             258.47
Name: 2, dtype: object
--------

In [33]:
2_979.35 + 903.25 + 2_838.55 + 3_278.85

10000.0

In [34]:
0 + 3843.75 + 903.25 + 2838.55 + 3278.85     

10864.4